# Decision Tree Extension

In [63]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import math
import random

# Decision Tree Implementation with Gini Index

In [64]:
class DecisionTree:

    def __init__(self, nbins, data_range):
        # Decision tree state here
        # Feel free to add methods
        self.bin_size = nbins
        self.range = data_range

    def preprocess(self, data):
        # Our dataset only has continuous data
        norm_data = np.clip((data - self.range[0]) / (self.range[1] - self.range[0]), 0, 1)
        categorical_data = np.floor(self.bin_size * norm_data).astype(int)
        return categorical_data

    def train(self, X, y):
        # training logic here
        # input is array of features and labels
        categorical_data = self.preprocess(X)
        feature_names = []
        for i in range(categorical_data.shape[1]):
            feature_names.append(i)
        X = list(categorical_data)
        y = list(y)
        self.tree = self.build_tree(X, y, [],feature_names)

    def predict(self, X):
        # Run model here
        # Return array of predictions where there is one prediction for each set of features
        categorical_data = self.preprocess(X)
        predict_results = []
        for x in categorical_data:
            label = self.get_label(self.tree, x)
            predict_results.append(label)
        result = np.array(predict_results)
        return result

    def gini(self, labels):
        class_count = {}
        for label in labels:
            if label not in class_count.keys():
                class_count[label] = 1
            else:
                class_count[label] += 1
        giniValue = 0
        for label in class_count.keys():
            prob = class_count[label] * 1.0 / len(labels)
            giniValue += prob * prob
        return 1 - giniValue

    def get_attribute_label(self, feature_column, attribute, labels):
        attri_labels = []
        for i in range(len(labels)):
            if feature_column[i] == attribute:
                attri_labels.append(labels[i])
        return attri_labels

    def get_attribute_rows(self, feature_column, feature_selected, attribute, features):
        new_features = []
        for i in range(len(features)):
            if feature_column[i] == attribute:
                new_features.append(features[i])
        if new_features != []:
            new_features = np.delete(new_features, feature_selected, 1)
        return new_features

    def get_gini(self, feature_column, labels):
        attributes = set(feature_column)
        feature_gini= 0
        for attribute in attributes:
            attri_labels = self.get_attribute_label(feature_column, attribute, labels)
            attri_count = feature_column.count(attribute)
            feature_gini += attri_count/len(feature_column) * self.gini(attri_labels)
        return feature_gini

    def get_majority(self, labels):
        class_count = {}
        for label in labels:
            if label not in class_count.keys():
                class_count[label] = 1
            else:
                class_count[label] += 1
        mostVote = -1000
        labelSelected = -1
        for label in class_count.keys():
            if class_count[label] > mostVote:
                mostVote = class_count[label]
                labelSelected = label
        return labelSelected

    def is_labels_all_same(self, labels):
        counter = labels.count(labels[0])
        if counter == len(labels):
            return True
        return False

    def is_row_all_same(self, X):
        return np.equal(X[0], X).all()

    def build_tree(self, X, y, parent_y, feature_names):
        if len(y) == 0:
            return self.get_majority(parent_y)

        if self.is_labels_all_same(y):
            return y[0]

        if len(feature_names) == 1 or self.is_row_all_same(X):
            return self.get_majority(y)

        min_gini = 1000
        selected = -1
        for i in range(len(feature_names)):
            feature_column = [x[i] for x in X]
            gini = self.get_gini(feature_column, y)
            if gini < min_gini:
                min_gini = gini
                selected = i

        best_feature = feature_names[selected]
        feature_column = [x[selected] for x in X]
        feature_names.remove(feature_names[selected])

        tree = {best_feature: {}}
        attributes = set(feature_column)

        for attribute in attributes:
            new_X = self.get_attribute_rows(feature_column, selected, attribute, X)
            new_y = self.get_attribute_label(feature_column, attribute, y)
            sub_feature_names = feature_names[:]
            tree[best_feature][attribute] = self.build_tree(new_X, new_y, y, sub_feature_names)
        return tree

    def get_label(self, tree, x):
        key_list = list(tree.keys())
        feature = key_list[0]
        predict = -1000
        for key in tree[feature].keys():
            if x[feature] == key:
                if type(tree[feature][key]).__name__ == 'dict':
                    predict = self.get_label(tree[feature][key], x)
                else:
                    predict = tree[feature][key]
        return predict


# Random Forest Implementation

In [65]:
def random_forest(X_train, y_train, X_test, y_test, num_bin, num_tree):
    num_feature = math.floor(2/3*len(X_train[0]))
    num_train = math.floor(2/3*len(X_train))
    feature_names = []
    train_used = []
    for i in range(len(X_train[0])):
        feature_names.append(i)

    for i in range(len(X_train)):
        train_used.append(i)

    results = []
    for i in range(num_tree):
        random.shuffle(feature_names)
        random.shuffle(train_used)
        feature_choose = feature_names[:num_feature]
        train_examples = train_used[:num_train]
        new_X_train = X_train[:, feature_choose]
        new_X_train = new_X_train[train_examples, :]
        new_y_train = y_train[train_examples]
        data_range = (new_X_train.min(0), new_X_train.max(0))
        tree = DecisionTree(num_bin, data_range)
        tree.train(new_X_train, new_y_train)
        new_X_test = X_test[:, feature_choose]
        predictions = tree.predict(new_X_test)
        results.append(predictions)
    
    results = np.array(results)
    solutions = []
    for i in range(len(results[0])):
        solution = get_majority(results[:, i])
        solutions.append(solution)

    solutions = np.array(solutions)
    y_labels = np.array(y_test)
    print("Random Forest Accuracy: ", evaluate(solutions, y_test))

# Evaluation Function

In [66]:
def evaluate(solutions, real):
    if(solutions.shape != real.shape):
        raise ValueError("Output is wrong shape.")
    predictions = np.array(solutions)
    labels = np.array(real)
    return (predictions == labels).sum() / float(labels.size)

def get_majority(labels):
    class_count = {}
    for label in labels:
        if label not in class_count.keys():
            class_count[label] = 1
        else:
            class_count[label] += 1
    mostVote = -1000
    labelSelected = -1
    for label in class_count.keys():
        if class_count[label] > mostVote:
            mostVote = class_count[label]
            labelSelected = label
    return labelSelected

# Test on Handwritten digit data

In [76]:
digits=load_digits()
X = digits.data
y = digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
print(X_train.shape)
print(y_train.shape)

(1078, 64)
(1078,)


## (1) Compare effects between single decision tree and random forest on sklearn

In [77]:
single_tree_sklearn = DecisionTreeClassifier(criterion="entropy")
single_tree_sklearn.fit(X_train, y_train)
sklearn_tree_predictions = single_tree_sklearn.predict(X_test)
print("sklearn Single Tree Accuracy: ", evaluate(sklearn_tree_predictions, y_test))

forest_sklearn = RandomForestClassifier(n_estimators=20)
forest_sklearn.fit(X_train, y_train)
sklearn_forest_predictions = forest_sklearn.predict(X_test)
print("sklearn Random Forest Accuracy: ", evaluate(sklearn_forest_predictions, y_test))

sklearn Single Tree Accuracy:  0.8442280945757997
sklearn Random Forest Accuracy:  0.9541029207232267


## (2) Compare effects between single decision tree and random forest on our implementation

In [70]:
data_range = (X_train.min(0), X_train.max(0))
single_tree = DecisionTree(3, data_range)
single_tree.train(X_train, y_train)
predictions = single_tree.predict(X_test)
print("Single Tree Accuracy: ", evaluate(predictions, y_test))

random_forest(X_train, y_train, X_test, y_test, 3, 20)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


Single Tree Accuracy:  0.6898470097357441
Random Forest Accuracy:  0.885952712100139


# Test on Spambase data

In [71]:
data = pd.read_csv("spambase.csv", delimiter=',')
data = data.sample(frac=1).reset_index(drop=True)
X_train = data.loc[:3000, "word1":"word57"].values
y_train = data.loc[:3000, "labels"].values
X_test = data.loc[3001:, "word1":"word57"].values
y_test = data.loc[3001:, "labels"].values

In [72]:
data_range = (X_train.min(0), X_train.max(0))
single_tree = DecisionTree(10, data_range)
single_tree.train(X_train, y_train)
predictions = single_tree.predict(X_test)
print("Single Tree Accuracy: ", evaluate(predictions, y_test))

random_forest(X_train, y_train, X_test, y_test, 10, 20)

Single Tree Accuracy:  0.766875
Random Forest Accuracy:  0.861875
